# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [1]:
finetuning = 1
inference = not finetuning

# Drive Mount

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# Read Data

In [3]:
import pandas as pd
data = pd.read_csv('v2(gender_code_change)_updated_hateSpeech.csv')
data

,Original Sentence,Converted Sentence,Name,original_lebel,Predicted Value for Original Sentence,Predicted Value for ner converted,Predict Biasness,Gender Masked,Gender and Name Masked
0,যত্তসব পাপন শালার ফাজলামী!!!!!,যত্তসব জাহান শালীর ফাজলামী!!!!!,['পাপন'],1,1,1,0,যত্তসব পাপন <Gender> ফাজলামী!!!!!,যত্তসব <Name> <Gender> ফাজলামী!!!!!
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,নাহার শালী রে রিমান্ডে নেওয়া দরকার,['পাপন'],1,1,1,0,পাপন <Gender> রে রিমান্ডে নেওয়া দরকার,<Name> <Gender> রে রিমান্ডে নেওয়া দরকার
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,নূর জাহান ম্যাডামের মেয়ে এতো বড় জারজ হবে এটা ...,"['জিল্লুর রহমান', 'তাহসিন']",1,1,1,0,জিল্লুর রহমান <Gender> <Gender> এতো বড় জারজ হব...,<Name> <Gender> <Gender> এতো বড় জারজ হবে এটা ...
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,শালী লুচ্চা দেখতে পাঠীর মত দেখা যায়,[],1,1,1,0,<Gender> লুচ্চা দেখতে পাঠার মত দেখা যায়,<Gender> লুচ্চা দেখতে পাঠার মত দেখা যায়
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,তুই তো শালী গাজা খাইছচ । তুর মার হেডায় খেলবে স...,['সাকিব'],1,1,1,0,তুই তো <Gender> গাজা খাইছচ।তুর মার হেডায় খেলবে...,তুই তো <Gender> গাজা খাইছচ।তুর মার হেডায় খেলবে...
...,...,...,...,...,...,...,...,...,...
10028,ভাই এরকম ভিডিও আরো করেন,আফা এরকম ভিডিও আরো করেন,[],0,0,0,0,<Gender> এরকম ভিডিও আরো করেন,<Gender> এরকম ভিডিও আরো করেন
10029,ভাই এরকম ভিডিও আরো করেন,বোন এরকম ভিডিও আরো করেন,[],0,0,0,0,<Gender> এরকম ভিডিও আরো করেন,<Gender> এরকম ভিডিও আরো করেন
10030,ভাই এরকম ভিডিও আরো করেন,ভাবী এরকম ভিডিও আরো করেন,[],0,0,0,0,<Gender> এরকম ভিডিও আরো করেন,<Gender> এরকম ভিডিও আরো করেন
10031,মূর্খ খালেদার ফাঁসি চাই,মূর্খা দুর্গা ফাঁসি চাই,['খালেদার'],0,0,0,0,<Gender> খালেদার ফাঁসি চাই,<Gender> <Name> ফাঁসি চাই


# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [4]:
temp = data[data['Predict Biasness']!= 1]
test = data[data['Predict Biasness']== 1]

In [5]:
t1 = temp[['Original Sentence', 'original_lebel']]
t2 = temp[['Converted Sentence', 'original_lebel']]
t2.columns = ['Original Sentence', 'original_lebel']
temp = pd.concat([t1, t2], axis = 0)
temp

,Original Sentence,original_lebel
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1
...,...,...
10028,আফা এরকম ভিডিও আরো করেন,0
10029,বোন এরকম ভিডিও আরো করেন,0
10030,ভাবী এরকম ভিডিও আরো করেন,0
10031,মূর্খা দুর্গা ফাঁসি চাই,0


In [6]:
validation = temp.groupby('original_lebel', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))
train = temp.drop(validation.index)

In [ ]:
train

,Original Sentence,original_lebel
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1
6,পাপনে পাগল হয়াছে,1
...,...,...
10027,কুলাঙ্গার সন্ত ান তুই ধ্বংস হয়ে যাবি তোর মায়...,0
10028,আফা এরকম ভিডিও আরো করেন,0
10029,বোন এরকম ভিডিও আরো করেন,0
10030,ভাবী এরকম ভিডিও আরো করেন,0


In [ ]:
validation

,Original Sentence,original_lebel
6495,এটা কোন ধরনের মজা ভাই,0
5711,পাগল হইলি না কি রে ভাই পালতু বিডিও দেস কেনো,0
8143,সাংবাদিক তুমি এখন নর নখএ জানতে চাও আর এতদিন কো...,0
6891,নুজহাত ভাবী আপনি আবার ছবি করেন আপনার মুভি গুল...,0
8978,আফা তুমি সেরা দিছেন ভইড়া 😂😂😂,0
...,...,...
1331,ছাগলি ইফতেখার ফাহমিকে সবচেয়ে বেশি ভরার কথা ছিল...,1
1478,"বাস্তবের সাথে মিল আছে , , আসলে এ ধরনের পুরুষর...",1
3400,এই শালার টুপি কই,1
2479,ডিসির বাল ছিরতে পারলি না এখন ছেলে টার পিছনে লা...,1


In [ ]:
test

,Original Sentence,Converted Sentence,Name,original_lebel,Predicted Value for Original Sentence,Predicted Value for ner converted,Predict Biasness,Gender Masked,Gender and Name Masked
8,দেখেছো মাগি না হিজরা বোঝা যাচ্ছে না। দেখে মনে ...,দেখেছো মাইগ্যা না হিজরা বোঝা যাচ্ছে না । দেখে...,[],1,1,0,1,দেখেছো <Gender> না হিজরা বোঝা যাচ্ছে না। দেখে ...,দেখেছো <Gender> না হিজরা বোঝা যাচ্ছে না। দেখে ...
32,মাগি খোর সাব্বির কে বাদ দাও,মাইগ্যা খোর সাব্বির কে বাদ দাও,['সাব্বির'],1,1,0,1,<Gender> খোর সাব্বির কে বাদ দাও,<Gender> খোর <Name> কে বাদ দাও
40,পাপন সাহেব উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ ...,গুলশান মেম উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ...,['পাপন'],1,0,1,1,পাপন <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গ...,<Name> <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে...
41,পাপন সাহেব উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ ...,আদৃতা বিবি উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ...,['পাপন'],1,0,1,1,পাপন <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গ...,<Name> <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে...
42,পাপন সাহেব উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ ...,সাবিত্রী সাহেবা উপর দিয়ে পানি ঢালে আর নিচ দিয়...,['পাপন'],1,0,1,1,পাপন <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গ...,<Name> <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে...
...,...,...,...,...,...,...,...,...,...
9716,সেই কাগজ কোথায় বেটা চাপা মেরে খাস দশ হাজার টাক...,সেই কাগজ কোথায় বেটি চাপা মেরে খাস দশ হাজার টাক...,[],0,1,0,1,সেই কাগজ কোথায় <Gender> চাপা মেরে খাস দশ হাজার...,সেই কাগজ কোথায় <Gender> চাপা মেরে খাস দশ হাজার...
9831,সব চোর হাসিনা দল হাসিনা হল ভোট চোর,সব চুন্নি সাদিয়া দল সাদিয়া হল ভোট চোর,"[' হাসিনা', 'হাসিনা']",0,0,1,1,সব <Gender> হাসিনা দল হাসিনা হল ভোট <Gender>,সব <Gender> <Name> দল <Name> হল ভোট <Gender>
9857,ও একটা পাগল,ও একটা পাগলিনী,[],0,0,1,1,ও একটা <Gender>,ও একটা <Gender>
9858,ও একটা পাগল,ও একটা পাগলী,[],0,0,1,1,ও একটা <Gender>,ও একটা <Gender>


# Finetune with ner + original all data

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model

c:\Users\MSI\anaconda4\envs\pytorch_gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model_path = "Hate-speech-CNERG/bengali-abusive-MuRIL"
tokenizer, b_model = load_transformer_based_model(model_path)

c:\Users\MSI\anaconda4\envs\pytorch_gpu\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MSI\.cache\huggingface\hub\models--Hate-speech-CNERG--bengali-abusive-MuRIL. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForMaskedLM were not initialized from the model checkpoint at Hate-

In [9]:
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

Total Pretrained tokens:  197285
Total number of tokens after adding the new ones:  197287


In [12]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm



class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0] #0th column all data

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long)
        except:
          print(index, self.train_csv[index])

dataset_train= BertDataset(train, tokenizer, max_length=200 )
dataset_val= BertDataset( validation, tokenizer, max_length=200)
dataset_test= BertDataset(test, tokenizer, max_length=200)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=16, shuffle=False)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle=False)


In [ ]:
id,am, tid, l = next(iter(train_loader))
x = b_model(id, am,tid)
x.hidden_states[0].shape

torch.Size([16, 200, 768])

In [13]:

class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 1)

    def forward(self,ids,mask,token_type_ids):
        o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
        o2= o2.max(dim=1).values
        o2 = self.dropout(o2)
        # print('------->>>>>>>>>',o2.values.shape)
        out= self.out(o2)

        return out, o2

masked_model=BERT(b_model)
# loss_fn = nn.BCEWithLogitsLoss()
# #Initialize Optimizer
# optimizer= optim.Adam(model.parameters(),lr= 0.0001)

In [ ]:
id,am, tid, l = next(iter(train_loader))
x,_ = masked_model(id, am,tid)
x.shape

torch.Size([16, 1])

In [15]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, loss):
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, loss):
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return torch.where(preds >= 0, 1, 0).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')


In [16]:
SAVE_MODEL = True
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.BCEWithLogitsLoss()
        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = '')
        m.begin_run(run, network, train_loader, validation_loader)
        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()
            for batch in tqdm(train_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)

                preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                labels = labels.type_as(preds).view(-1, 1)
                # print(labels.shape)
                # print('----------')
                # print(preds.shape)


                loss = loss_fn(preds, labels)
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
                optimizer.step()
                scheduler.step()


                m.track_loss(loss)
                m.track_num_correct(preds, labels)


            network.eval()
            for batch in tqdm(validation_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                with torch.no_grad():
                    preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                    labels = labels.type_as(preds).view(-1, 1)
                    loss = loss_fn(preds, labels)
                    m.track_validation_loss(loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'baseline_HateSpeech.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return


if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = 16, fresh_training = True)

,run,epoch,loss,accuracy,validation loss,validation accuracy,epoch duration,run duration,lr,batch_size,device
0,1,1,0.028892,0.801904,0.019666,0.885091,309.217926,309.430127,0.0001,16,cuda
1,1,2,0.013839,0.916872,0.015515,0.906127,309.257331,632.943277,0.0001,16,cuda
2,1,3,0.008679,0.951912,0.013399,0.922956,309.334175,956.572774,0.0001,16,cuda
3,1,4,0.005686,0.971853,0.012693,0.929266,306.017982,1276.905110,0.0001,16,cuda
4,1,5,0.003808,0.982767,0.012636,0.928478,302.741740,1593.583041,0.0001,16,cuda
5,1,6,0.002536,0.990399,0.012339,0.931633,308.495611,1916.031832,0.0001,16,cuda
6,1,7,0.001875,0.992450,0.012509,0.933474,309.885362,2239.977961,0.0001,16,cuda
7,1,8,0.001336,0.995323,0.013124,0.932948,309.972326,2563.984028,0.0001,16,cuda
8,1,9,0.001025,0.996389,0.012900,0.934788,310.003501,2888.030280,0.0001,16,cuda
9,1,10,0.000818,0.997538,0.013296,0.934788,303.422712,3205.492740,0.0001,16,cuda


Saved model and optimizer at baseline_HateSpeech.pth


In [ ]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return


if inference:
  load_model(masked_model, None, PATH = '/content/drive/MyDrive/#Research/# GB/saved_model/baseline_toxicity.pth')

Loaded model from /content/drive/MyDrive/#Research/# GB/saved_model/Senti_Masked.pth


In [17]:

class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0] #0th column all data

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])

def get_prediction(data_loader, network):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids, token_type_ids, attention_masks, labels = batch
    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        preds, hidden_states = network(input_ids, attention_masks,token_type_ids)
        labels = labels.type_as(preds).view(-1, 1)
        preds =  torch.where(preds >= 0, 1, 0)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()
dataset_test= BertDataset(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)

  0%|          | 0/33 [00:00<?, ?it/s]c:\Users\MSI\anaconda4\envs\pytorch_gpu\lib\site-packages\transformers\tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 33/33 [00:08<00:00,  4.01it/s]


In [18]:
test['pred'] = [x[0] for pred in p for x in pred.tolist()]
test

C:\Users\MSI\AppData\Local\Temp\ipykernel_14132\339689748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = [x[0] for pred in p for x in pred.tolist()]


,Original Sentence,Converted Sentence,Name,original_lebel,Predicted Value for Original Sentence,Predicted Value for ner converted,Predict Biasness,Gender Masked,Gender and Name Masked,pred
8,দেখেছো মাগি না হিজরা বোঝা যাচ্ছে না। দেখে মনে ...,দেখেছো মাইগ্যা না হিজরা বোঝা যাচ্ছে না । দেখে...,[],1,1,0,1,দেখেছো <Gender> না হিজরা বোঝা যাচ্ছে না। দেখে ...,দেখেছো <Gender> না হিজরা বোঝা যাচ্ছে না। দেখে ...,1
32,মাগি খোর সাব্বির কে বাদ দাও,মাইগ্যা খোর সাব্বির কে বাদ দাও,['সাব্বির'],1,1,0,1,<Gender> খোর সাব্বির কে বাদ দাও,<Gender> খোর <Name> কে বাদ দাও,1
40,পাপন সাহেব উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ ...,গুলশান মেম উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ...,['পাপন'],1,0,1,1,পাপন <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গ...,<Name> <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে...,0
41,পাপন সাহেব উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ ...,আদৃতা বিবি উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ...,['পাপন'],1,0,1,1,পাপন <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গ...,<Name> <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে...,0
42,পাপন সাহেব উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গাছ ...,সাবিত্রী সাহেবা উপর দিয়ে পানি ঢালে আর নিচ দিয়...,['পাপন'],1,0,1,1,পাপন <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে গ...,<Name> <Gender> উপর দিয়ে পানি ঢালে আর নিচ দিয়ে...,0
...,...,...,...,...,...,...,...,...,...,...
9716,সেই কাগজ কোথায় বেটা চাপা মেরে খাস দশ হাজার টাক...,সেই কাগজ কোথায় বেটি চাপা মেরে খাস দশ হাজার টাক...,[],0,1,0,1,সেই কাগজ কোথায় <Gender> চাপা মেরে খাস দশ হাজার...,সেই কাগজ কোথায় <Gender> চাপা মেরে খাস দশ হাজার...,1
9831,সব চোর হাসিনা দল হাসিনা হল ভোট চোর,সব চুন্নি সাদিয়া দল সাদিয়া হল ভোট চোর,"[' হাসিনা', 'হাসিনা']",0,0,1,1,সব <Gender> হাসিনা দল হাসিনা হল ভোট <Gender>,সব <Gender> <Name> দল <Name> হল ভোট <Gender>,0
9857,ও একটা পাগল,ও একটা পাগলিনী,[],0,0,1,1,ও একটা <Gender>,ও একটা <Gender>,1
9858,ও একটা পাগল,ও একটা পাগলী,[],0,0,1,1,ও একটা <Gender>,ও একটা <Gender>,1


Result

In [19]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  525 Total mismatch:  109 Accuracy:  0.7923809523809524


In [20]:
# Before fine tuning
sum(data['Predicted Value for Original Sentence'] != data['Predicted Value for ner converted'])

525


# Prediction of NER Converted text data

In [21]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm


class BertDatasetNER(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDatasetNER, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,1] #1 column for NER Converted

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])
dataset_test= BertDatasetNER(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)
test['pred_ner'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['original_lebel'] != test['pred_ner'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

  0%|          | 0/33 [00:00<?, ?it/s]c:\Users\MSI\anaconda4\envs\pytorch_gpu\lib\site-packages\transformers\tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 33/33 [00:08<00:00,  4.08it/s]
C:\Users\MSI\AppData\Local\Temp\ipykernel_14132\530226218.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_ne

Total test data:  525 Total mismatch:  103 Accuracy:  0.8038095238095238 Bias:  36


In [22]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  525 Total mismatch:  103 Accuracy:  0.8038095238095238 Bias:  36


In [23]:
test.to_csv('baseline_hateSpeech_reSult.csv', index = False)